# LABORATORIO # 4

In [18]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Descripcion de los datos

In [19]:
# Cargar el conjunto de datos
data = pd.read_csv('train.csv')

In [20]:
# Información general del conjunto de datos
print(data.info())

# Estadísticas descriptivas
print(data.describe())

# Valores únicos en 'target'
print(data['target'].value_counts())

# Cantidad de palabras clave y ubicaciones únicas
print(data['keyword'].nunique())
print(data['location'].nunique())

# Valores nulos por columna
print(data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None
                 id      target
count   7613.000000  7613.00000
mean    5441.934848     0.42966
std     3137.116090     0.49506
min        1.000000     0.00000
25%     2734.000000     0.00000
50%     5408.000000     0.00000
75%     8146.000000     1.00000
max    10873.000000     1.00000
target
0    4342
1    3271
Name: count, dtype: int64
221
3341
id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


In [21]:
# Crear un CountVectorizer para unigramas y bigramas
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words='english')

# Ajustar y transformar el texto preprocesado
X = vectorizer.fit_transform(data['text'])

# Obtener los nombres de las características (unigramas y bigramas)
feature_names = vectorizer.get_feature_names_out()

# Crear un DataFrame para visualizar los unigramas y bigramas más frecuentes
n_grams_df = pd.DataFrame(X.toarray(), columns=feature_names)

# Calcular la suma de los unigramas y bigramas para obtener los más comunes
most_common_unigrams = n_grams_df.sum().sort_values(ascending=False).head(10)
most_common_bigrams = n_grams_df.sum(axis=0).sort_values(ascending=False).head(10)

print("Unigramas más comunes:")
print(most_common_unigrams)

print("\nBigramas más comunes:")
print(most_common_bigrams)


Unigramas más comunes:
http       4309
https       411
û_          348
like        348
amp         344
just        323
new         226
û_ http     217
people      200
news        197
dtype: int64

Bigramas más comunes:
http       4309
https       411
û_          348
like        348
amp         344
just        323
new         226
û_ http     217
people      200
news        197
dtype: int64


In [22]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2, random_state=42)

# Crear un TfidfVectorizer para transformar el texto en características numéricas
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Crear y entrenar un modelo Naive Bayes
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test_vec)

# Evaluar el modelo
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Exactitud del modelo:", accuracy)
print("\nInforme de clasificación:\n", report)

Exactitud del modelo: 0.7997373604727511

Informe de clasificación:
               precision    recall  f1-score   support

           0       0.77      0.93      0.84       874
           1       0.86      0.63      0.73       649

    accuracy                           0.80      1523
   macro avg       0.82      0.78      0.78      1523
weighted avg       0.81      0.80      0.79      1523



In [23]:
# # Convertir el texto a minúsculas
# data['text'] = data['text'].str.lower()

# # Quitar caracteres especiales como #, @ y apóstrofes
# data['text'] = data['text'].apply(lambda text: re.sub(r'[@#’]', '', text))

# # Quitar las URLs
# data['text'] = data['text'].apply(lambda text: re.sub(r'http\S+', '', text))

# # Quitar emoticones
# data['text'] = data['text'].apply(lambda text: re.sub(r'[^\w\s]', '', text))

# # Quitar signos de puntuación
# data['text'] = data['text'].apply(lambda text: re.sub(r'[^\w\s]', '', text))

# # Tokenizar el texto
# data['text'] = data['text'].apply(word_tokenize)

# # Definir lista de stopwords en español
# stop_words = set(stopwords.words('spanish'))

# # Quitar stopwords
# data['text'] = data['text'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

# # Quitar números excepto "911"
# data['text'] = data['text'].apply(lambda tokens: [word for word in tokens if word == '911' or not word.isdigit()])

# # Unir tokens de nuevo en texto
# data['text'] = data['text'].apply(lambda tokens: ' '.join(tokens))

# # Guardar el conjunto de datos preprocesado
# data.to_csv('preprocessed_data.csv', index=False)
